## langchain.chains.OpenApiEndpointChain

This seems to be a slightly more advanced API calling utility, which reads an OpenAPI spec. You can define "operations" from endpoints, but I'm not sure how to use multiple operations with planning, or if this is possible.

In [1]:
from langchain.tools import OpenAPISpec, APIOperation
from langchain.chains import OpenAPIEndpointChain
from langchain.requests import Requests
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec

import dotenv
import pprint

dotenv.load_dotenv()

pp = pprint.PrettyPrinter(indent=4)

Read the spec, create operations for endpoints, create a new chain.

In [2]:
spec = OpenAPISpec.from_file("openapi.json")

search_op = APIOperation.from_openapi_spec(spec, '/search', "get")
#d2p_op = APIOperation.from_openapi_spec(spec, '/disease-phenotypes', "get") # not used

llm = ChatOpenAI(model = "gpt-4") # Load a Language Model

chain = OpenAPIEndpointChain.from_api_operation(
    search_op, 
    llm, 
    requests=Requests(), 
    verbose=True,
    return_intermediate_steps=True # Return request and response text
)


Attempting to load an OpenAPI 3.0.2 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


Run it:

In [3]:
%%time
output = chain("What phenotypes are associated with Cystic Fibrosis?")



> Entering new OpenAPIEndpointChain chain...


> Entering new APIRequesterChain chain...
Prompt after formatting:
You are a helpful AI Assistant. Please provide JSON arguments to agentFunc() based on the user's instructions.

API_SCHEMA: ```typescript
/* Search for entities in the Monarch knowledge graph */
type search_entity = (_: {
/* The ontology term to search for. */
		term: string,
/* A single category to search within as a string. Valid categories are: biolink:Disease, biolink:PhenotypicQuality, and biolink:Gene */
		category?: string,
/* The maximum number of search results to return. */
		limit?: number,
/* Offset for pagination of results */
		offset?: number,
}) => any;
```

USER_INSTRUCTIONS: "What phenotypes are associated with Cystic Fibrosis?"

Your arguments must be plain json provided in a markdown block:

ARGS: ```json
{valid json conforming to API_SCHEMA}
```

Example
-----

ARGS: ```json
{"foo": "bar", "baz": {"qux": "quux"}}
```

The block must be no more than 1 

Notice in the below output there is some gene information that is included from the search results (it defaulted to 10 results).

In [4]:
pp.pprint(output)

{   'instructions': 'What phenotypes are associated with Cystic Fibrosis?',
    'intermediate_steps': {   'request_args': '{"term": "Cystic Fibrosis", '
                                              '"category": '
                                              '"biolink:PhenotypicQuality", '
                                              '"limit": 10, "offset": 0}',
                              'response_text': '{"results":[{"id":"MONDO:0009061","name":"cystic '
                                               'fibrosis","categories":["biolink:Disease"],"description":"Cystic '
                                               'fibrosis (CF) is a genetic '
                                               'disorder characterized by the '
                                               'production of sweat with a '
                                               'high salt content and mucus '
                                               'secretions with an abnormal '
                             